# CAPSTONE PART 1 #


### Goal: Creating functions for converting all variety of audio recordings, be them recorded from the microphone or digital audio files, into a NumPy-array of digital samples. ###

In [9]:
import numpy as np 
import librosa
from typing import Tuple, Union, Callable
import pyaudio

In [10]:
def load_audio_file(filespath):
    """
    Load an audio file and return the audio samples and sample rate.

    Args:
        filepath (str): Path to the audio file.

    Returns:
        Tuple[np.ndarray, int]: A tuple containing the audio samples as a NumPy array
        and the sample rate of the audio file.
    """
    #this depends on whether we have pre-recorded audio or not, otherwise this would change
    samples, sample_rate = librosa.load(filespath, sr=None) 
    return samples, sample_rate

In [11]:
#shows the analog signal at the given time frames
def analog_signal(times: np.ndarray)  -> np.ndarray:
    """
    Generate an analog signal based on the given times.

    Args:
        times (np.ndarray): An array of time values,wqhich would then be converted into a digital sample.

    Returns:
        the val of the analog signal at a given time 
    """
    return 1 / (1 + np.exp(-10 * (times - 1)))
    

In [12]:
#direct from the analogtodigital notebook
def temporal_sampler(
    signal: Callable[[np.ndarray], np.ndarray], *, duration: float, sampling_rate: float
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Extracts samples from an analog signal according to the specified sampling rate,
    returning the times and the corresponding samples extracted at those times.

    Parameters
    ----------
    signal : Callable[[ndarray], ndarray]
        Another Python function (i.e. a "callable"), which behaves like f(t)
        and accepts a time value (in seconds) as an input and returns a
        measurement (e.g. in volts) as an output. You can expect this to behave like
        a vectorized function i.e. it can be passed a NumPy-array of input times
        and it will return a corresponding array of measurements.

    duration : float
        The duration of the signal, specified in seconds (a non-negative float)

    sampling_rate : float
        The sampling rate specified in Hertz.

    Returns
    -------
    (times, samples) : Tuple[ndarray, ndarray]
        The shape-(N,) array of times and the corresponding shape-(N,) array
        samples extracted from the analog signal

    """
    N_samples = np.floor(sampling_rate * duration) + 1

    # shape-(N,) array of times at which we sample the analog signal
    times = np.arange(N_samples) * (1 / sampling_rate)  # seconds

    # shape-(N,) array of samples extracted from the analog signal
    samples = signal(times)

    return times, samples


In [13]:
def quantize(samples: np.ndarray, bit_depth: int) -> np.ndarray:
    """
    Given an array of N samples and a bit-depth of M, return the array of
    quantized samples derived from the domain [samples.min(), samples.max()]
    that has been quantized into 2**M evenly-spaced values.

    Parameters
    ----------
    samples : numpy.ndarray, shape-(N,)
        An array of N samples

    bit_depth: int
        The bit-depth, M, used to quantize the samples among
        2**M evenly spaced values spanning [samples.min(), samples.max()].

    Returns
    -------
    quantized_samples : numpy.ndarray, shape-(N,)
        The corresponding array where each sample has been replaced
        by the nearest quantized value

    """

    # spanning [samples.min(), samples.max()]
    quantized_values = np.linspace(samples.min(), samples.max(), 2 ** bit_depth)

    abs_differences = np.abs(samples[:, np.newaxis] - quantized_values)

  
    bin_lookup = np.argmin(abs_differences, axis=1)

    return quantized_values[bin_lookup]

In [14]:
def analog_to_digital(
    analog_signal: Callable[[np.ndarray], np.ndarray],
    *,
    sampling_rate: float,
    bit_depth: int,
    duration: float
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Digitizes a given analog signal based on desired sampling rate and bit-depth.
    
    Parameters
    ----------
    analog_signal : Callable[[ndarray], ndarray]
        Another Python function, f(t), which accepts a time value (in seconds) as
        an input and returns a measurement (in volts) as an output.
    
    sampling_rate : float
        The sampling rate specified in Hertz.
    
    bit_depth: int
        The bit-depth, M, used to quantize the samples among
        2**M evenly spaced values spanning [samples.min(), samples.max()].
    
    duration : float
        The duration of the signal, specified in seconds (a non-negative float).
    
    Returns
    -------
    (times, digital_signal) : Tuple[ndarray, ndarray]
        The shape-(N,) array of times and the corresponding
        shape-(N,) array representing the sampled & quantized digital signal.
    """

    times, samples = temporal_sampler(
        analog_signal, duration=duration, sampling_rate=sampling_rate
    )
    digital_signal = quantize(samples, bit_depth)

    return times, digital_signal

### TESTING ###

In [16]:
#song path will be altered post test, just to see if it works 
#DIGITAL AUDIO FILES
song_path = r"C:\Users\abhyu\dev\CogWorks\Week_1\data\trumpet.wav"
samples, sample_rate = load_audio_file(song_path)
print(samples)

# ANALOG SIGNAL


duration = 2  # seconds, however depends on how long the sample length is, just simulating
sample_rate = 10  # Hz, again simulating a sampling rate
bit_depth = 2  # bits
times, samples = analog_to_digital(
    analog_signal=analog_signal,
    duration=duration,
    sampling_rate=sample_rate,
    bit_depth=bit_depth,
)
print(samples)

[-2.000e+00 -5.000e+00 -1.000e+00 ...  4.225e+03  4.414e+03  4.397e+03]
[4.53978687e-05 4.53978687e-05 4.53978687e-05 4.53978687e-05
 4.53978687e-05 4.53978687e-05 4.53978687e-05 4.53978687e-05
 4.53978687e-05 3.33348466e-01 3.33348466e-01 6.66651534e-01
 9.99954602e-01 9.99954602e-01 9.99954602e-01 9.99954602e-01
 9.99954602e-01 9.99954602e-01 9.99954602e-01 9.99954602e-01
 9.99954602e-01]
